# 天元 MegEngine 快速上手

注意：本教程假定你具备一定的 Python 编程基础，并了解深度学习的基础概念。

我们将向你介绍使用天元 MegEngine 实现的完整的机器学习工作流程，以便你快速地熟悉常见 API 的使用方式。

请先运行下面的代码，检验你的环境中是否已经安装好 MegEngine（[访问官网安装教程](https://megengine.org.cn/install/)）：

In [1]:
import megengine

print(megengine.__version__)

1.3.0


接下来我们将通过 MNIST 手写数字识别的案例帮助你快速上手天元 MegEngine 的使用。

## 数据的加载和预处理

数据的加载和预处理往往会耗费大量的精力， MegEngine 提供了一系列接口来规范化这些处理工作：

- [megengine.data.dataset.Dataset](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.data.dataset.Dataset.html) 是 MegEngine 中表示数据集的抽象类，存储样本和相应的标签；
- [megengine.data.DataLoader](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.data.DataLoader.html) 负责根据传入的 `Dataset` 等参数生成一个可迭代的对象。

在 [megengine.data.dataset](https://megengine.org.cn/doc/stable/zh/reference/data.html#dataset) 模块中为用户提供了非常多经典的数据集，比如本次教程中用到的 [MNIST](http://yann.lecun.com/exdb/mnist/) 数据集：

In [2]:
from megengine.data.dataset import MNIST

# 如果使用 MegStudio 环境，请将 MNIST_DATA_PATH 为 /home/megstudio/dataset/MNIST/
MNIST_DATA_PATH = "/data/datasets/MNIST/"

# 获取训练数据集，如果本地没有数据集，请将 download 参数设置为 True
train_dataset = MNIST(root=MNIST_DATA_PATH, train=True, download=False)
test_dataset = MNIST(root=MNIST_DATA_PATH, train=False, download=False)

02 01:49:55 process the raw files of train set...
100%|████████████████████████████████| 60000/60000 [00:00<00:00, 1738115.31it/s]
02 01:49:57 process the raw files of test set...
100%|████████████████████████████████| 10000/10000 [00:00<00:00, 1693641.83it/s]


对于如何加载自定义的 `Dataset` ，请参考用户指南 利用 Dataset 封装一个数据集。

将 `Dataset` 作为参数传给 `DataLoader` 时，我们还需要为其指定数据预处理和抽样逻辑：

- [megengine.data.transfrom](https://megengine.org.cn/doc/stable/zh/reference/data.html#transform) 提供了常见的数据变换操作，作为预处理手段，支持 `Compose` 组合；
- [megengine.data.sampler](https://megengine.org.cn/doc/stable/zh/reference/data.html#sampler) 提供了常见的采样方法，如顺序采样和随机采样等，可指定 ``batch_size`` 参数。

In [3]:
from megengine.data import DataLoader
from megengine.data.transform import ToMode, Pad, Normalize, Compose
from megengine.data.sampler import RandomSampler, SequentialSampler

batch_size=64

# 创建 Sampler
train_sampler = RandomSampler(train_dataset, batch_size=batch_size)
test_sampler = SequentialSampler(test_dataset, batch_size=batch_size)

# 数据预处理方式
transform = Compose([
    Normalize(mean=0.1307*255, std=0.3081*255),
    Pad(2),
    ToMode('CHW'),
])

# 创建 Dataloader
train_dataloader = DataLoader(train_dataset, train_sampler, transform)
test_dataloader = dataloader_test = DataLoader(test_dataset, test_sampler, transform)

In [4]:
for idx, (X, y) in enumerate(test_dataloader):
    print("Index: ", idx)
    print("Shape of X: ", X.shape) # [N, C, H, W]
    print("Shape of y: ", y.shape, y.dtype)
    break

Index:  0
Shape of X:  (64, 1, 32, 32)
Shape of y:  (64,) int32


## 定义网络结构

在 MegEngine 中定义网络最常见的方式是创建一个继承自 [megengine.module.Module](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.module.Module.html) 的类，接着：

- 在 `__init__` 方法中定义网络的层，各类算子可以在 [megengine.functional](https://megengine.org.cn/doc/stable/zh/reference/functional.html) 和 [megengine.module](https://megengine.org.cn/doc/stable/zh/reference/module.html) 中找到；
- 通过 `forward` 方法描述数据通过网络前向传播时依次执行的算子，这些算子和它们之间的依赖关系也就定义了网络的结构。

当检测到 GPU 环境时，MegEngine 会自动将相应的计算过程在 GPU 中执行（无需手动指定设备）从而实现加速。

In [5]:
import megengine.module as M
import megengine.functional as F

# 定义网络
class Net(M.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = M.Conv2d(1, 20, kernel_size=5, bias=False)
        self.bn0 = M.BatchNorm2d(20)
        self.relu0 = M.ReLU()
        self.pool0 = M.MaxPool2d(2)
        self.conv1 = M.Conv2d(20, 20, kernel_size=5, bias=False)
        self.bn1 = M.BatchNorm2d(20)
        self.relu1 = M.ReLU()
        self.pool1 = M.MaxPool2d(2)
        self.fc0 = M.Linear(500, 64, bias=True)
        self.relu2 = M.ReLU()
        self.fc1 = M.Linear(64, 10, bias=True)

    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.pool0(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = F.flatten(x, 1)
        x = self.fc0(x)
        x = self.relu2(x)
        x = self.fc1(x)
        return x
    
# 实例化网络
net = Net()
print(net)

Net(
  (conv0): Conv2d(1, 20, kernel_size=(5, 5), bias=False)
  (bn0): BatchNorm2d(20, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0)
  (conv1): Conv2d(20, 20, kernel_size=(5, 5), bias=False)
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0)
  (fc0): Linear(in_features=500, out_features=64, bias=True)
  (relu2): ReLU()
  (fc1): Linear(in_features=64, out_features=10, bias=True)
)


## 定义损失函数、优化器

为了实现对模型的训练（即对模型中参数的优化），我们还需要定义：

- 损失函数（Loss Function），大部分常见的损失函数实现在 [megengine.function.loss](https://megengine.org.cn/doc/stable/zh/reference/functional.html#loss) 模块中；
- 优化器（Optimizer），常见的优化器实现在 [megengine.optimizer](https://megengine.org.cn/doc/stable/zh/reference/optimizer.html), 支持不同的学习率、权重衰减等等优化策略；
- MegEngine 的自动求导功能由 [megengine.autodiff](https://megengine.org.cn/doc/stable/zh/reference/autodiff.html) 模块实现，其中 [GradManager](https://megengine.org.cn/doc/stable/zh/reference/autodiff.html#gradmanager) 负责管理梯度。

In [6]:
from megengine.optimizer import SGD
from megengine.autodiff import GradManager

optimizer = SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

gm = GradManager().attach(net.parameters())

## 模型训练

在 MegEngine 中计算图默认为动态图模式，可通过 [megengine.jit.trace](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.jit.trace.html) 非常便捷地将动态图转为静态图。

In [7]:
from megengine.jit import trace

@trace(symbolic=True)
def train_func(data, label, *, gm, net):
    net.train()
    with gm:
        pred = net(data)
        loss = F.loss.cross_entropy(pred, label)
        gm.backward(loss)
    return pred, loss

我们设置 10 个训练周期，整个过程中将对训练数据集分批次进行预测，根据反向传播算法更新模型的参数。

In [8]:
import numpy as np
import megengine as mge

total_epochs = 10
for epoch in range(total_epochs):
    total_loss = 0
    for step, (batch_data, batch_label) in enumerate(train_dataloader):
        batch_label = batch_label.astype(np.int32)
        pred, loss = train_func(mge.tensor(batch_data), mge.tensor(batch_label), gm=gm, net=net)
        optimizer.step().clear_grad()
        total_loss += loss.numpy().item()
    print("epoch: {}, loss {}".format(epoch, total_loss/len(train_dataloader)))

epoch: 0, loss 0.13873870883669173
epoch: 1, loss 0.05043510790826924
epoch: 2, loss 0.03717820324114899
epoch: 3, loss 0.029249643720289282
epoch: 4, loss 0.025445997079552363
epoch: 5, loss 0.02158043481139486
epoch: 6, loss 0.018581382247176505
epoch: 7, loss 0.01729959516382929
epoch: 8, loss 0.015273293452476388
epoch: 9, loss 0.014109803669488252


## 模型的保存与加载

在 MegEngine 中通过使用 [megengine.save](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.save.html) 和 [megengine.load](https://megengine.org.cn/doc/stable/zh/reference/api/megengine.load.html) 进行模型的保存与加载。

我们首先将训练好的模型的保存到本地：

In [9]:
mge.save(net.state_dict(), 'mnist_net.mge')

接着我们可以加载本地的模型文件，在测试集上进行预测，以检测模型的性能。

In [10]:
net = Net()
state_dict = mge.load('mnist_net.mge')
net.load_state_dict(state_dict)

# 在测试集上验证模型性能，注意此时要使用 .eval()
@trace(symbolic=True)
def eval_func(data, label, *, net):
    net.eval()
    pred = net(data)
    loss = F.loss.cross_entropy(pred, label)
    return pred, loss

correct = 0
total = 0
for idx, (batch_data, batch_label) in enumerate(test_dataloader):
    batch_label = batch_label.astype(np.int32)
    pred, loss = eval_func(mge.tensor(batch_data), mge.tensor(batch_label), net=net)
    predicted = pred.numpy().argmax(axis=1)
    correct += (predicted == batch_label).sum().item()
    total += batch_label.shape[0]
print("correct: {}, total: {}, accuracy: {}".format(correct, total, float(correct) / total))

correct: 9919, total: 10000, accuracy: 0.9919
